In [18]:
from PyPDF2 import PdfFileWriter, PdfFileReader
import requests
from bs4 import BeautifulSoup
import pdfplumber
import re
import pandas as pd
import datetime

def parse_report_by_url(url):
    response = requests.get(url)
    file = open("tmp/daily_slides.pdf", "wb")
    file.write(response.content)
    file.close()

    inputpdf = PdfFileReader(open("tmp/daily_slides.pdf", "rb"))
    output = PdfFileWriter()
    output.addPage(inputpdf.getPage(2))
    with open("tmp/manufacturer_table.pdf", "wb") as outputStream:
        output.write(outputStream)
    with pdfplumber.open("tmp/manufacturer_table.pdf") as pdf:
        first_page = pdf.pages[0]
        text = first_page.extract_text()
        while text.find('  ') != -1:
            text = text.replace('  ', ' ')
        text = text.replace(' ,', '')
        text = text.replace(', ', '')
        text = text.replace(',', '')
        lines = text.split('\n')      
    return lines

def search_manufacturer(report):
    items = []
    #Huge brain search. Any better suggestion?
    q = [i for i, item in enumerate(report) if re.search('จาแนกตามบ', item.replace(" ", ""))]
    if(len(q)==0):
        return False
    q = q[0]
    for data in report[q+1:q+4]:
        data = data.replace(" ราย","")
        data = data.split(" ")
        items.append((data[0], int(data[1])))
    items = dict(items)
    #Huge brain test case
    if ('Sinovac' not in items or items['Sinovac'] < 5000000):
        return False
    else:
        return items

def calculate_rate(df):
    first_date = pd.to_datetime('2021-07-02')
    last_date = pd.to_datetime(df.iloc[-1]['date'])
    loop_date = first_date
    df['date'] = pd.to_datetime(df['date'])
    while (loop_date <= last_date):
        date_data = df[df['date'] == loop_date]
        if (len(date_data) == 0):
            previous_data = df[(pd.to_datetime(df['date']) == loop_date-pd.DateOffset(1))]
            previous_data['date'] = loop_date
            df=df.append(previous_data,ignore_index=True)    
        loop_date = loop_date + pd.DateOffset(1)
    df=df.sort_values(by=['date']).reset_index(drop=True)
    old_df = df[df['date']<'2021-07-02']
    new_df = df[df['date']>='2021-07-02']
    new_df['AstraZeneca_rate'] = new_df['AstraZeneca'].diff()
    new_df['Sinopharm_rate'] = new_df['Sinopharm'].diff()
    new_df['Sinovac_rate'] = new_df['Sinovac'].diff()
    new_df = new_df.fillna(0)
    return old_df.append(new_df, ignore_index=True)


In [21]:
df = pd.read_json("../../components/gis/data/vaccine-manufacturer-timeseries.json")
if len(df)>0:
    latest_date = pd.to_datetime(df.iloc[-1]['date'])
else:
    latest_date = pd.to_datetime("2021-07-02")
url = "https://ddc.moph.go.th/vaccine-covid19/diaryPresentMonth/0" + str(latest_date.month) + "/10/2021"
req = requests.get(url)
soup = BeautifulSoup(req.content, 'html.parser')
manufacturer_timeseries = df
rows = soup.find_all("td", text=re.compile('สไลด์นำเสนอ'))
for row in rows[latest_date.day:len(rows)]:
    tr = row.parent
    report_url = tr.find("a").get("href")
    print('--'+tr.text.strip()+'--')
    print(report_url)
    report = parse_report_by_url(report_url)
    data = search_manufacturer(report)
    print(data)
    latest_date += datetime.timedelta(days=1)
    if data != False :
        data["date"] = latest_date
        manufacturer_timeseries = manufacturer_timeseries.append(data,ignore_index=True)

manufacturer_timeseries = calculate_rate(manufacturer_timeseries)
manufacturer_timeseries['date']=manufacturer_timeseries['date'].astype(str)

manufacturer_timeseries.to_json("../../components/gis/data/vaccine-manufacturer-timeseries.json",orient="records",indent=2)

print('saved!')

--สไลด์นำเสนอวัคซีน ประจำวันที่ 2 กรกฎาคม 2564--
https://ddc.moph.go.th/vaccine-covid19/getFiles/10/1625222017817.pdf
{'Sinovac': 6403172, 'AstraZeneca': 3766181, 'Sinopharm': 57830}
--สไลด์นำเสนอวัคซีน ประจำวันที่  3 กรกฎาคม 2564--
https://ddc.moph.go.th/vaccine-covid19/getFiles/10/1625377983773.pdf
{'Sinovac': 6493669, 'AstraZeneca': 4000698, 'Sinopharm': 77925}
--สไลด์นำเสนอวัคซีน ประจำวันที่  4 กรกฎาคม 2564--
https://ddc.moph.go.th/vaccine-covid19/getFiles/10/1625378146400.pdf
{'Sinovac': 6513839, 'AstraZeneca': 4061982, 'Sinopharm': 95076}
--สไลด์นำเสนอวัคซีน ประจำวันที่  5 กรกฎาคม 2564--
https://ddc.moph.go.th/vaccine-covid19/getFiles/10/1625546085529.pdf
{'Sinovac': 6539664, 'AstraZeneca': 4124197, 'Sinopharm': 113887}
--สไลด์นำเสนอวัคซีน ประจำวันที่  6 กรกฎาคม 2564--
https://ddc.moph.go.th/vaccine-covid19/getFiles/10/1625565016860.pdf
{'Sinovac': 6619009, 'AstraZeneca': 4306880, 'Sinopharm': 132501}
--สไลด์นำเสนอวัคซีน ประจำวันที่  7  กรกฎาคม 2564--
https://ddc.moph.go.th/vacci

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy [frame.py:3607]


[14]
